In [1]:
import matplotlib.pylab as plt
import pandas as pd
import torch.nn as nn
import numpy
from sklearn import preprocessing

In [2]:
dataset = pd.read_csv(filepath_or_buffer='spambase/spambase.data', header = None)
inputs_indx = [x + 1 for x in range(56)]
label_indx = 57
dataset.head()


,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [3]:
scaler = preprocessing.StandardScaler()
dataset.loc[:, inputs_indx] = scaler.fit_transform(dataset.loc[:, inputs_indx])
print(scaler.mean_)
dataset.head()

[2.13014562e-01 2.80656379e-01 6.54249076e-02 3.12223430e-01
 9.59008911e-02 1.14207781e-01 1.05294501e-01 9.00673767e-02
 2.39413171e-01 5.98239513e-02 5.41701804e-01 9.39295805e-02
 5.86263856e-02 4.92045208e-02 2.48848077e-01 1.42586394e-01
 1.84744621e-01 1.66209954e+00 8.55770485e-02 8.09760922e-01
 1.21201913e-01 1.01645295e-01 9.42686373e-02 5.49504456e-01
 2.65383612e-01 7.67304934e-01 1.24844599e-01 9.89154532e-02
 1.02851554e-01 6.47533145e-02 4.70484677e-02 9.72288633e-02
 4.78352532e-02 1.05411867e-01 9.74766355e-02 1.36952836e-01
 1.32014779e-02 7.86285590e-02 6.48337318e-02 4.36665942e-02
 1.32338622e-01 4.60986742e-02 7.91958270e-02 3.01223647e-01
 1.79823951e-01 5.44446859e-03 3.18691589e-02 3.85746577e-02
 1.39030428e-01 1.69758748e-02 2.69070854e-01 7.58106933e-02
 4.42382091e-02 5.19151511e+00 5.21727885e+01 2.83289285e+02]


,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.330885,0.712859,-0.0469,0.011565,-0.350266,-0.291794,-0.262562,-0.323302,-0.371364,...,-0.158453,-0.514307,-0.155198,0.624007,-0.308355,-0.103048,-0.045247,0.045298,-0.008724,1
1,0.21,0.051909,0.435130,-0.0469,-0.256117,0.672399,0.244743,-0.088010,-0.323302,1.086711,...,-0.158453,-0.026007,-0.155198,0.126203,0.423783,0.008763,-0.002443,0.250563,1.228324,1
2,0.06,-0.165072,0.851723,-0.0469,1.364846,0.343685,0.193644,0.036670,1.974017,0.016422,...,-0.117376,0.014684,-0.155198,0.008496,0.440053,-0.079754,0.145921,2.221106,3.258733,1
3,0.00,-0.165072,-0.556761,-0.0469,0.472573,-0.350266,0.500237,1.308402,0.789462,0.605857,...,-0.158453,-0.007511,-0.155198,-0.161934,-0.308355,-0.103048,-0.052150,-0.062466,-0.152222,1
4,0.00,-0.165072,-0.556761,-0.0469,0.472573,-0.350266,0.500237,1.308402,0.789462,0.605857,...,-0.158453,-0.014910,-0.155198,-0.164387,-0.308355,-0.103048,-0.052150,-0.062466,-0.152222,1
